In [ ]:
from huggingface_hub import login

login()

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import json
import re

pipe = pipeline(
    "text-generation",
    model="Qwen/Qwen3-4B-Instruct-2507",
    max_new_tokens=256,
    temperature=0.1,   # low temp = more consistent JSON
    device=0,
    return_full_text=False  # Don't include the prompt in output
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
def extract_book_filters(text: str):
    prompt = f"""
You extract book search filters from natural language.

Return ONLY valid JSON using exactly these keys:
- "title": string or null
- "author": string or null
- "categories": array of strings
- "time_period": string or null
- "language": string or null
- "audience": string or null
- "format": string or null

Be careful on language and categories. A classic Russian novel should have categories including "Russian Literature" and not language "Russian".
If something is not mentioned, set it to null (or [] for arrays).

Respond with JSON ONLY. No explanations. No markdown.

User request:
\"\"\"{text}\"\"\"
"""

    result = pipe(prompt)[0]["generated_text"]

    # Extract JSON from markdown code blocks or raw JSON
    try:
        # Remove markdown code blocks if present
        json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', result, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
        else:
            # Try to find raw JSON
            json_start = result.find("{")
            json_end = result.rfind("}") + 1
            json_str = result[json_start:json_end]
        
        return json.loads(json_str)
    except Exception as e:
        print("Failed to parse JSON:", e)
        print("Raw model output:", result)
        return None


In [8]:
test_queries = [
    {"id": 1, "input": "I want a book set in medieval times, written by J.R.R. Tolkien."},
    {"id": 2, "input": "Find me a sci-fi novel by Isaac Asimov."},
    {"id": 3, "input": "I'm looking for a romance story that takes place in Victorian England."},
    {"id": 4, "input": "Give me a mystery thriller, any author is fine."},
    {"id": 5, "input": "Show me a book about dragons in a fantasy world."},
    {"id": 6, "input": "I want something written in German, preferably historical fiction."},
    {"id": 7, "input": "Recommend a children's book about animals."},
    {"id": 8, "input": "A classic Russian novel please."},
    {"id": 9, "input": "Find an audiobook version of a modern detective story."},
    {"id": 10, "input": "Looking for a dystopian book, maybe by George Orwell?"},
    {"id": 11, "input": "Give me something medieval or renaissance themed, doesn't matter the author."},
    {"id": 12, "input": "Books by Brandon Sanderson, preferably fantasy."},
    {"id": 13, "input": "I want an epic sci-fi saga set in the far future."},
    {"id": 14, "input": "Find me a short story written in Spanish."},
    {"id": 15, "input": "Show me books suitable for young adults dealing with coming-of-age themes."},
    {"id": 16, "input": "Anything from Stephen King but not horror."},
    {"id": 17, "input": "I'd like a medieval fantasy romance."},
    {"id": 18, "input": "Looking for historical biographies, 18th century preferred."},
    {"id": 19, "input": "Something set during World War II but fiction, not non-fiction."},
    {"id": 20, "input": "Give me magical realism originally published in Latin America."},
    {"id": 21, "input": "Show me a book written before the Renaissance, preferably mythology."},
    {"id": 22, "input": "I need a contemporary romance written by a female author."},
    {"id": 23, "input": "Find poetry collections in French."},
    {"id": 24, "input": "A comedic sci-fi book, audiobook format if possible."},
    {"id": 25, "input": "Something post-apocalyptic, similar to The Road but not by McCarthy."},
    {"id": 26, "input": "Give me a book set in ancient Rome."},
    {"id": 27, "input": "Looking for a graphic novel, superhero-themed."},
    {"id": 28, "input": "I want a philosophical novel that deals with existentialism."},
    {"id": 29, "input": "Show me a western adventure with cowboys."},
    {"id": 30, "input": "Find a cyberpunk book by a Japanese author."},
    {"id": 31, "input": "Something for kids about space exploration."},
    {"id": 32, "input": "A high-fantasy epic with elves and magic."},
    {"id": 33, "input": "Find a medieval Japanese historical novel."},
    {"id": 34, "input": "Looking for a romantic comedy, any language is fine."},
    {"id": 35, "input": "Give me a war novel written by Ernest Hemingway."},
    {"id": 36, "input": "Show me anything that fits the noir detective genre."},
    {"id": 37, "input": "Books written in Italian set during the Renaissance."},
    {"id": 38, "input": "Find feminist literature from the 20th century."},
    {"id": 39, "input": "I want a book about artificial intelligence, preferably non-fiction."},
    {"id": 40, "input": "Give me crime fiction with strong female leads."},
    {"id": 41, "input": "Show me mythology retellings written recently."},
    {"id": 42, "input": "A classic adventure novel in audiobook format."},
    {"id": 43, "input": "Looking for dystopian YA books."},
    {"id": 44, "input": "Recommend a Chinese wuxia novel."},
    {"id": 45, "input": "Anything that takes place during the Bronze Age."},
    {"id": 46, "input": "Give me a fantasy novel inspired by Scandinavian folklore."},
    {"id": 47, "input": "Looking for a novel with LGBTQ+ themes."},
    {"id": 48, "input": "Find comedy books, preferably British humor."},
    {"id": 49, "input": "A spy thriller set during the Cold War."},
    {"id": 50, "input": "I want a science textbook written in English."}
]

In [6]:
test_queries = [{"id": 3, "input": "I'm looking for a romance story that takes place in Victorian England."}]

In [9]:
for query in test_queries:
    user_input = query["input"]
    filters = extract_book_filters(user_input)
    print(f"Query {query['id']}: {user_input}")
    print(f"Filters: {filters}")

Query 1: I want a book set in medieval times, written by J.R.R. Tolkien.
Filters: {'title': None, 'author': 'J.R.R. Tolkien', 'categories': ['Medieval Fiction'], 'time_period': 'medieval times', 'language': None, 'audience': None, 'format': None}
Query 2: Find me a sci-fi novel by Isaac Asimov.
Filters: {'title': None, 'author': 'Isaac Asimov', 'categories': [], 'time_period': None, 'language': None, 'audience': None, 'format': None}
Query 3: I'm looking for a romance story that takes place in Victorian England.
Filters: {'title': None, 'author': None, 'categories': ['Romance', 'Victorian England'], 'time_period': None, 'language': None, 'audience': None, 'format': None}
Query 4: Give me a mystery thriller, any author is fine.
Filters: {'title': None, 'author': None, 'categories': ['Mystery', 'Thriller'], 'time_period': None, 'language': None, 'audience': None, 'format': None}
Query 5: Show me a book about dragons in a fantasy world.
Filters: {'title': None, 'author': None, 'categories

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Query 9: Find an audiobook version of a modern detective story.
Filters: {'title': None, 'author': None, 'categories': ['Detective Fiction'], 'time_period': 'modern', 'language': None, 'audience': None, 'format': 'audiobook'}
Query 10: Looking for a dystopian book, maybe by George Orwell?
Filters: {'title': None, 'author': 'George Orwell', 'categories': ['Dystopian Fiction'], 'time_period': None, 'language': None, 'audience': None, 'format': None}
Query 11: Give me something medieval or renaissance themed, doesn't matter the author.
Filters: {'title': None, 'author': None, 'categories': ['Medieval', 'Renaissance'], 'time_period': None, 'language': None, 'audience': None, 'format': None}
Query 12: Books by Brandon Sanderson, preferably fantasy.
Filters: {'title': None, 'author': 'Brandon Sanderson', 'categories': ['Fantasy'], 'time_period': None, 'language': None, 'audience': None, 'format': None}
Query 13: I want an epic sci-fi saga set in the far future.
Filters: {'title': None, 'auth